<a href="https://colab.research.google.com/github/nguyenanhtienabcd/AIO2024_EXERCISE/blob/feature%2FMODULE8-WEEK2-project2/m08w02_project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**dataset description:** SQuAD2.0 là một bộ data hướng đọc hiểu , bao gồm các loại văn khác nhiêu về nhiều chủ đề, ứng với mỗi đoạn văn sẽ có các câu hỏi ngăn kèm theo. \\
Câu trả lời cho các đoạn văn là những cụm từ có sẵn trong đoạn căn cho trước. hoặc các câu trả lời được dựa vào đoạn văn \\

**Reader**: DistilBERT đầu tiên ta sẽ xây dựng model Reader hay chinh slaf model QA

In [1]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
!pip install -qq accelerate == 0.26.1
!apt install git-lfs
!pip install --upgrade transformers accelerate
!pip install -U evaluate datasets


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.35.2 which is incompatible.
ERROR: Invalid requirement: '==': Expected package name at the start of dependency specifier
    ==
    ^
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found

In [2]:
!pip install -qq peft==0.4.0  # Install or upgrade peft to a compatible version

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import numpy as np
from tqdm.auto import tqdm
import collections

import torch

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [5]:
# setup configuration
MOLDEL_NAME = "distilbert-base-uncased"
MAX_LENGTH = 384
STRIDE = 128

In [6]:
# download dataset từ hugging face
DATASET_NAME = 'squad_V2'
raw_datasets = load_dataset(DATASET_NAME)

# load tokenizer để run một số sample
tokenizer = AutoTokenizer.from_pretrained(MOLDEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

Hàm preprocess_training_examples nhận dữ liệu đào tạo làm đầu vào và tiền xử lý để chuẩn bị cho việc huấn luyện mô hình hỏi đáp trong quá trình này, hàm trích xuất danh sách câu hỏi, mã hóa thông tin đầu vào bằng tokenizer, và trích xuất offset_mapping vá sample_map để ánh xạ vị trí từ mã hóa về văn bản gốc. Hàm cũng xác định vị trí bắt đầu và kết thúc của câu trả lời trong ngữ cảnh và thêm thông tin về vị trí này vào biến input.

In [7]:
def preprocess_training_examples(examples):
    # Trích xuất danh sách câu hỏi từ các ví dụ và
    # loại bỏ khoảng trắng ở đầu và cuối
    questions = [q.strip() for q in examples["question"]]

    # Token hóa dữ liệu đầu vào bằng cách sử dụng tokenizer
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Trích xuất offset_mapping từ inputs và xóa nó khỏi inputs
    offset_mapping = inputs.pop("offset_mapping")

    # Trích xuất sample_map từ inputs và xóa nó khỏi inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")

    # Trích xuất thông tin trả lời từ các ví dụ
    answers = examples["answers"]

    # Tạo danh sách để lưu trữ vị trí bắt đầu và kết thúc của các câu trả lời
    start_positions = []
    end_positions = []

    # Duyệt qua offset mapping để tìm vị trí bắt đầu và kết thúc của các câu trả lời
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]

        # Trích xuất sequence_ids từ inputs
        sequence_ids = inputs.sequence_ids(i)

        # Xác định vị trí bắt đầu và kết thúc của ngữ cảnh
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Trích xuất thông tin câu trả lời cho mẫu này
        answer = answers[sample_idx]

        if len(answer["text"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Xác định vị trí ký tự bắt đầu và kết thúc của câu trả lời trong ngữ cảnh
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])

            # Nếu câu trả lời không nằm hoàn toàn trong ngữ cảnh, gán nhận là (0, 0)
            if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Nếu không, gán vị trí bắt đầu và kết thúc dựa trên vị trí của các mã thông tin
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)
                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

    # Thêm thông tin vị trí bắt đầu và kết thúc vào inputs
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs


In [8]:
# Tạo một biến train_dataset và gán cho nó giá trị sau khi áp dụng
# hàm preprocess_training_examples lên tập dữ liệu "train"
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,  # Bật chế độ xử lý theo từng batch bằng cách đặt batched=True
    remove_columns=raw_datasets["train"].column_names,  # Loại bỏ các cột không cần thiết trong tập dữ liệu "train"
)

# In ra độ dài của tập dữ liệu "train" ban đầu và
# độ dài của tập dữ liệu đã được xử lý (train_dataset)
len(raw_datasets["train"]), len(train_dataset)


(130319, 131754)

**Tokenize valset**: làm tương tự cho tệp val, thực hiện việc tiền xử lý dữ liệu cho việc đánh giá mô hình. Hàm chuẩn bị danh sách câu hỏi, mã hóa các câu hỏi vàn văn bản liên quan bằng cách sử dụng tikenizer. Sau đó xác dịnh ví dụ tham chiếu cho từng dòng đầu vào và điều chỉnh ánh xạ offset để loại vỏ các offset không phù hợp với sequence_ids. cuối cùng là thêm thông tin về ví dụ tham chiếu đầu vào.

In [9]:
def preprocess_validation_examples(examples):
  # chuẩn bị danh sách câu hỏi bằng cách
  # loại bỏ khoảng trắng ở đầu và cuối câu hỏi
  questions = [q.strip() for q in examples["question"]]

  # mã hóa các câu hỏi và văn bản liên quan
  inputs = tokenizer(
      questions,
      examples['context'],
      max_length=MAX_LENGTH,
      truncation="only_second",
      stride=STRIDE,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

  # lấy ánh xạ lại để ánh xạ ví dụ tham chiếu cho từng dòng trong inputs
  # Lấy ánh xạ để ánh xạ lại ví dụ tham chiếu cho từng dòng trong inputs
  sample_map = inputs.pop("overflow_to_sample_mapping")
  example_ids = []

  # Xác định ví dụ tham chiếu cho mỗi dòng đầu vào và điều chỉnh ánh xạ offset
  for i in range(len(inputs["input_ids"])):
      sample_idx = sample_map[i]
      example_ids.append(examples["id"][sample_idx])

      sequence_ids = inputs.sequence_ids(i)
      offset = inputs["offset_mapping"][i]

      # Loại bỏ các offset không phù hợp với sequence_ids
      inputs["offset_mapping"][i] = [
          o if sequence_ids[k] == 1 else None \
          for k, o in enumerate(offset)
      ]

  # Thêm thông tin ví dụ tham chiếu vào đầu vào
  inputs["example_id"] = example_ids

  return inputs


In [10]:
# Tạo một biến validation_dataset và gán giá trị bằng việc sử dụng dữ liệu
# từ raw_datasets[\"validation\"] sau khi áp dụng một hàm xử lý trước.
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

# In ra độ dài của raw_datasets[\"validation\"]
# và validation_dataset để so sánh.
len(raw_datasets["validation"]), len(validation_dataset)


Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

(11873, 12134)

In [11]:
# train model

model = AutoModelForQuestionAnswering.from_pretrained(MOLDEL_NAME)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Tạo đối tượng args là các tham số cho quá trình huấn luyện
args = TrainingArguments(
    output_dir="distilbert-finetuned-squadv2",  # Thư mục lưu output
    evaluation_strategy="no",  # Chế độ đánh giá không tự động sau mỗi epoch
    save_strategy="epoch",  # Lưu checkpoint sau mỗi epoch
    learning_rate=2e-5,  # Tốc độ học
    num_train_epochs=3,  # Số epoch huấn luyện
    weight_decay=0.01,  # Giảm trọng lượng mô hình để tránh overfitting
    fp16=True,  # Sử dụng kiểu dữ liệu half-precision để tối ưu tài nguyên
    push_to_hub=True,  # Đẩy kết quả huấn luyện lên HuggingFace Hub
    # Remove or comment out the dispatch_batches argument
    # dispatch_batches=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Khởi tạo một đối tượng Trainer để huấn luyện mô hình
trainer = Trainer(
    model=model,  # Sử dụng mô hình đã tạo trước đó
    args=args,  # Các tham số và cấu hình huấn luyện
    train_dataset=train_dataset,  # Sử dụng tập dữ liệu huấn luyện
    eval_dataset=validation_dataset,  # Sử dụng tập dữ liệu đánh giá
    tokenizer=tokenizer,  # Sử dụng tokenizer để xử lý văn bản
)

# Bắt đầu quá trình huấn luyện


<ipython-input-13-3b4ad20cb793>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nguyenanhtienbk1996 (nguyenanhtienbk1996-AAA) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,3.128400
1000,2.157800
1500,1.891000
2000,1.781200
2500,1.696000
3000,1.651500
3500,1.557000
4000,1.537900
4500,1.512600
5000,1.509800


Step,Training Loss
500,3.128400
1000,2.157800
1500,1.891000
2000,1.781200
2500,1.696000
3000,1.651500
3500,1.557000
4000,1.537900
4500,1.512600
5000,1.509800


TrainOutput(global_step=49410, training_loss=1.021572342335852, metrics={'train_runtime': 5037.2409, 'train_samples_per_second': 78.468, 'train_steps_per_second': 9.809, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.021572342335852, 'epoch': 3.0})

In [16]:
trainer.push_to_hub(commit_message="Training complete")


events.out.tfevents.1740190381.96a31c6e5764.198.0:   0%|          | 0.00/26.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tienabcd/distilbert-finetuned-squadv2/commit/29fcefafd129729a3cb3c9f666976face7c496b0', commit_message='Training complete', commit_description='', oid='29fcefafd129729a3cb3c9f666976face7c496b0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tienabcd/distilbert-finetuned-squadv2', endpoint='https://huggingface.co', repo_type='model', repo_id='tienabcd/distilbert-finetuned-squadv2'), pr_revision=None, pr_num=None)

In [14]:
import evaluate
metric = evaluate.load("squad_v2")

In [23]:
N_BEST = 20  # Số lượng kết quả tốt nhất được lựa chọn sau khi dự đoán
MAX_ANS_LENGTH = 30  # Độ dài tối đa cho câu trả lời dự đoán

def compute_metrics(start_logits, end_logits, features, examples):
    # Tạo một từ điển mặc định để ánh xạ mỗi ví dụ
    example_to_features = collections.defaultdict(list)
    # với danh sách các đặc trưng tương ứng
    for idx, feature in enumerate(features):
        example_to_features[feature['example_id']].append(idx)

    predicted_answers = []
    # Use tqdm.tqdm to create a progress bar
    for example in tqdm.tqdm(examples): # Call tqdm
        example_id = example['id']
        context = example['context']
        answers = []

        # Lặp qua tất cả các đặc trưng liên quan đến ví dụ đó
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]['offset_mapping']

            # Lấy các chỉ số có giá trị lớn nhất cho start và end logits
            start_indexes = np.argsort(start_logit)[-1:-N_BEST-1:-1].tolist()
            end_indexes = np.argsort(end_logit)[-1:-N_BEST-1:-1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Bỏ qua các câu trả lời
                    # không hoàn toàn nằm trong ngữ cảnh
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Bỏ qua các câu trả lời có độ dài >
                    # max_answer_length
                    if end_index - start_index + 1 > MAX_ANS_LENGTH:
                        continue

                    # Tạo một câu trả lời mới
                    text = context[offsets[start_index][0]:offsets[end_index][1]]
                    logit_score = start_logit[start_index] + end_logit[end_index]
                    answer = {
                        'text': text,
                        'logit_score': logit_score,
                    }
                    answers.append(answer)

        # Chọn câu trả lời có điểm số tốt nhất
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x['logit_score'])
            answer_dict = {
                'id': example_id,
                'prediction_text': best_answer['text'],
                'no_answer_probability': 1 - best_answer['logit_score']
            }
        else:
            answer_dict = {
                'id': example_id,
                'prediction_text': '',
                'no_answer_probability': 1.0
            }
        predicted_answers.append(answer_dict)

    # Tạo danh sách câu trả lời lý thuyết từ các ví dụ
    theoretical_answers = [
        {'id': ex['id'], 'answers': ex['answers']} for ex in examples
    ]

    # Sử dụng metric.compute để tính toán các độ đo và trả về kết quả
    return metric.compute(
        predictions=predicted_answers,
        references=theoretical_answers
    )

In [24]:
# Thực hiện dự đoán trên tập dữ liệu validation
predictions, _, _ = trainer.predict(validation_dataset)

# Lấy ra thông tin về các điểm bắt đầu và
# điểm kết thúc của câu trả lời dự đoán
start_logits, end_logits = predictions

# Tính toán các chỉ số đánh giá sử dụng hàm compute_metrics
results = compute_metrics(
    start_logits,
    end_logits,
    validation_dataset,
    raw_datasets["validation"]
)
results

  0%|          | 0/11873 [00:00<?, ?it/s]

{'exact': 35.803924871557314,
 'f1': 36.84249262950221,
 'total': 11873,
 'HasAns_exact': 0.10121457489878542,
 'HasAns_f1': 2.181328439622195,
 'HasAns_total': 5928,
 'NoAns_exact': 71.40454163162322,
 'NoAns_f1': 71.40454163162322,
 'NoAns_total': 5945,
 'best_exact': 50.07159100480081,
 'best_exact_thresh': 0.0,
 'best_f1': 50.07159100480081,
 'best_f1_thresh': 0.0}

In [28]:
# Use a pipeline as a high-level helper
from transformers import pipeline

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'tienabcd/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

# Test the model with an example
INPUT_QUESTION = 'where do i live?'
INPUT_CONTEXT = 'i live in Hanoi, the capital of Vietnam.'
result = pipe(question=INPUT_QUESTION, context=INPUT_CONTEXT)

print(result)
# Output: {'score': 0.97179114818573, 'start': 11, 'end': 21, 'answer': 'AI Vietnam'}


Device set to use cuda:0


{'score': 0.4770520031452179, 'start': 10, 'end': 15, 'answer': 'Hanoi'}


## 2. Checking code

In [ ]:
t_contexts = raw_datasets['train']['context']
t_contexts[0]

In [ ]:
t_questions = raw_datasets['train']['question']
t_questions[0]

In [ ]:
t_answers = raw_datasets['train']['answers']
t_answers[0]

In [ ]:
t_questions[0].strip()

In [ ]:
t_input = tokenizer(
    t_questions[0],
    t_contexts[0],
    max_length=MAX_LENGTH,
    truncation="only_second",
    stride=STRIDE,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
)
t_input

In [ ]:
t_offset_mapping = t_input.pop("offset_mapping")
print(t_offset_mapping)

In [ ]:
t_sample_map = t_input.pop("overflow_to_sample_mapping")
print(t_sample_map)

In [ ]:
# 0 (Câu hỏi)
# 1 (Context)
print(t_input.sequence_ids(0))

In [ ]:
t_sample_map[0]

In [ ]:
len(t_answers[0]["text"][0])

In [ ]:
offset = t_offset_mapping
offset[0][9]

In [ ]:
start_positions = []
end_positions = []

i = 0
offset = t_offset_mapping[i]

sample_idx = t_sample_map[i]

# Trích xuất sequence_ids từ inputs
sequence_ids = t_input.sequence_ids(i)

# Xác định vị trí bắt đầu và kết thúc của ngữ cảnh
idx = 0
while sequence_ids[idx] != 1:
    idx += 1
context_start = idx
print('context_start:', context_start)
while sequence_ids[idx] == 1:
    idx += 1
context_end = idx - 1
print('context_end:', context_end)

# Trích xuất thông tin câu trả lời cho mẫu này
t_answer_1 = t_answers[sample_idx] #answers[0]

if len(t_answer_1["text"]) == 0:
    start_positions.append(0)
    end_positions.append(0)
else:
    # Xác định vị trí ký tự bắt đầu và kết thúc của câu trả lời trong ngữ cảnh
    start_char = t_answer_1["answer_start"][0]
    print('start_char:', start_char)
    end_char = t_answer_1["answer_start"][0] + len(t_answer_1["text"][0])
    print('end_char:', end_char)

    #----
    print('offset[context_start]:',offset[context_start])
    print('offset[context_end][1]:', offset[context_end])
    #----

    # Nếu câu trả lời không nằm hoàn toàn trong ngữ cảnh, gán nhận là (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:

        start_positions.append(0)
        print('start_positions 1:', start_positions)
        end_positions.append(0)
        print('end_positions 1:', end_positions)
    else:
        # Nếu không, gán vị trí bắt đầu và kết thúc dựa trên vị trí của các mã thông tin
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)
        print('start_positions 2:', start_positions)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)
        print('end_positions 2:', end_positions)